# Measure of the actin fusion focus components'distribution 


#### Folders architecture: 
For each strain, fluorescence profiles for each protein should be kept in a common folder. File name should include 'profile' and the name of the protein the fluorescent profile describe.  

#### Package required:
-matplotlib

-numpy

-pandas

-numpy

-lmfit

#### Inputs required:
-path: path to the folder containing the strains folders to analyze, plots will be saved in that directory
-strain: list of strain subfolders to compare
-colors: list of colors for plots
-poi_subset: list of proteins of interest included in the analysis
-scale: image scale (in px/µm)

In [ ]:
path= '/mnt/c/Users/thomasv/Documents/Signal-size-ft/'
strain=['wt','strain1'] 
colors=['paleturquoise','wheat','navajowhite']
poi_subset=['POI1','POI2']
scale=15.4321

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import pandas as pd
import os
from trajalign.traj import Traj
from trajalign.average import load_directory
import lmfit
from lmfit.lineshapes import gaussian

In [ ]:
#create dictionnary associating color to strains
palette={}
for i in range(len(strain)): 
    palette[strain[i]]=colors[i]

# get list of profiles presents in strains'subfolders
profiles={}
for i in range(len(strain)):
    temp=os.listdir(os.path.join(path,strain[i]))
    profiles[strain[i]]=[ file for file in temp if 'profile' in file ]

# Align data and extract measurements

Note: based on gaussian fitting, oter fitting can be considered if necessary. In case of noisy signal, smoothing curves can help. 

In [ ]:
#function to fit gaussian on each profile and align center fit at 0, and extract FWHM and estimated height gaussian for each profile
def gauss_alignment(path):
    #load fluoresence profile
    df=pd.read_csv(path)
    df.columns=("distance","fluorescence")
        
    #find max to align
    y=(df.fluorescence-min(df.fluorescence))/(max(df.fluorescence)-min(df.fluorescence))
    x=df.distance
    model = lmfit.models.GaussianModel()
    pars = model.guess(y,x)
    result = model.fit(y, x=x, params=pars)
   
    centered_x=(x-result.params['center'].value)/scale #get center gaussian fit and align on that (in micrometers)
    fwhm=result.params['fwhm'].value/scale #get full width at half max (FWHM) gaussian fit
    point=fwhm/2 #get half distance FWHM = distance half maximum from center 

    y2=df.fluorescence
    model2 = lmfit.models.GaussianModel()
    pars2 = model2.guess(y2,x)
    result2 = model2.fit(y2, x=x, params=pars2)
    
    est_height=result2.params['height'].value
    data=pd.DataFrame({'dist_to_fusion':centered_x,
                            'norm_fluorescence':y,    # fluorescence as fitted through gaussian 
                          'fluorescence':df.fluorescence}) 
        
    return data,fwhm,point, est_height

In [ ]:
# apply gauss_alignment function on all profiles 

aligned_profiles_gaussian={}
points={}
FWHM={}
Est_fluo={}
for i in strain:
    Fw={}
    est_f={}
    al_prof={}
    pt={}
    for j in profiles[i]:
        name=j.split('_')
        poi=j.split('_')[3].split('.')[0]
        print(os.path.join(path,i,j))
        df_gaussian,F,p,h=gauss_alignment(os.path.join(path,i,j))
        if F!= 'None':
            Fw[poi]=Fw.get(poi, [])+[F]
            est_f[poi]=est_f.get(poi, [])+[h]
            al_prof[poi]=al_prof.get(poi, [])+[df_gaussian]
            pt[poi]=pt.get(poi, [])+[p]
        if F== 'None':
            al_prof[poi]=al_prof.get(poi, [])+[df_gaussian]
            Fw[poi]=Fw.get(poi, [])+[0]
            
    aligned_profiles_gaussian[i]=al_prof
    FWHM[i]=Fw
    Est_fluo[i]=est_f
    points[i]=pt
    Fw={}
    al_prof={}
    pt={}

In [ ]:
#for visually checking that alignment worked correctly 
colors=['red','lime', 'darkturquoise','hotpink','forestgreen','coral', 'purple','aliceblue','orchid','indigo','mediumvioletred','deeppink','darkorange','goldenrod','grey','maroon','mediumblue','dodgerblue']
for i in strain:
    for j in aligned_profiles_gaussian[i].keys():
        plt.figure(figsize=(35,20))
        n=0
        for k in range(len(aligned_profiles_gaussian[i][j])):
            plt.plot(aligned_profiles_gaussian[i][j][k].dist_to_fusion, aligned_profiles_gaussian[i][j][k].fluorescence,color=colors[n], marker='o')
            plt.title('strain:'+str(i)+' poi: '+str(j))
            n+=1
            if n==len(colors):
                n=0
        plt.show()

# Plot Full Width at Half Maximum (FWHM)

In [ ]:
#create dataframe to scatter plot FWHM (full width at half maximum)
dfs = list()
for st in strain:
    for poi in poi_subset:
        if poi in FWHM[st].keys():
            df=pd.DataFrame({'FWHM':FWHM[st][poi]})
            df['Strain'] = st
            df['Protein'] = poi
            df['Colors']=palette[st]
            dfs.append(df)
dfs = pd.concat(dfs)
dfs.Strain = dfs.Strain.astype('str')
dfs.Protein = dfs.Protein.astype('str')

In [ ]:
#plot FWHM as scatter plot for each protein of interest
for poi in poi_subset:
    colList=[]
    for color in dfs.Colors[dfs.Protein==poi]:
        if color not in colList:
            colList.append(color)

    sn.set_style("whitegrid")
    ax=plt.figure(figsize=(2,6))
    df=dfs[dfs.Protein==poi]
    x = "Strain"
    y = "FWHM"
    ax = sn.swarmplot(data=df, x=x, y=y,hue=x, palette=colList,size=5) 
    df_mean = df.groupby('Strain', sort=False)['FWHM'].median()
    _ = [ax.hlines(y, i-.1, i+.1, zorder=5,color='black',alpha=1) for i, y in df_mean.reset_index()['FWHM'].items()]
    df_quant25 = df.groupby('Strain', sort=False)['FWHM'].quantile(0.25)
    df_quant75 = df.groupby('Strain', sort=False)['FWHM'].quantile(0.75)
    index=list(range(0,len(strain)))
    df_quant=zip(index,list(df_quant25),list(df_quant75))
    _ = [ax.hlines(y, i-.05, i+.05, zorder=3,color='black',alpha=1) for i, y in df_quant25.reset_index()['FWHM'].items()]
    _ = [ax.hlines(y, i-.05, i+.05, zorder=3,color='black',alpha=1) for i, y in df_quant75.reset_index()['FWHM'].items()]
    _ = [ax.vlines(i, y,z, zorder=3,color='black',alpha=1) for i, y,z in df_quant]
    plt.xticks(rotation=45, ha="right")
    ax.set(ylim=(0, 2))
    plt.legend([],[], frameon=False)
    plt.title(poi)
    plt.savefig(os.path.join(path,'FWHM-comparison'+poi+'.pdf'))

# FWHM ratio

To compute the ratios,  profiles from the two proteins that need to be compared should have been extracted from the same cell.

For ratio: POI_A and POI_B are specified so that the FWHM ratio will be POI_A / POI_B

In [ ]:
POI_A= 'POI1'
POI_B= 'POI2'

for st in strain:
    FWHM[st]['ratio']=[]
    for i in range (len (FWHM[st][POI_B])): 
        ratio=FWHM[st][POI_A][i]/ FWHM[st][POI_B][i]
        FWHM[st]['ratio']=FWHM[st].get('ratio', [])+[ratio]

In [ ]:
ct=0
r_subset= ['ratio','norm']

dfr = list()
for st in strain:
    df=pd.DataFrame({'Ratio':FWHM[st]['ratio']})
    df['Strain'] = st
    df['Type'] = 'ratio'
    df['Colors']=palette[st]
    dfr.append(df)
dfr = pd.concat(dfr)
dfr.Strain = dfr.Strain.astype('str')
dfr.Type = dfr.Type.astype('str')


colList=[]
for color in dfr.Colors:
    if color not in colList:
        colList.append(color)

    
sn.set_style("whitegrid")
ax=plt.figure(figsize=(5,8))
d=dfr
x = "Strain"
y = "Ratio"
ax = sn.swarmplot(data=d, x=x, y=y,hue=x, palette=colList,size=5)
df_mean = d.groupby('Strain', sort=False)['Ratio'].mean()
_ = [ax.hlines(y, i-.1, i+.1, zorder=5,color='black',alpha=1) for i, y in df_mean.reset_index()['Ratio'].items()]
df_quant25 = d.groupby('Strain', sort=False)['Ratio'].quantile(0.25)
df_quant75 = d.groupby('Strain', sort=False)['Ratio'].quantile(0.75)
index=list(range(0,len(strain)))
df_quant=zip(index,list(df_quant25),list(df_quant75))
_ = [ax.hlines(y, i-.05, i+.05, zorder=3,color='black',alpha=1) for i, y in df_quant25.reset_index()['Ratio'].items()]
_ = [ax.hlines(y, i-.05, i+.05, zorder=3,color='black',alpha=1) for i, y in df_quant75.reset_index()['Ratio'].items()]
_ = [ax.vlines(i, y,z, zorder=3,color='black',alpha=1) for i, y,z in df_quant]
plt.xticks(rotation=45, ha="right")
plt.legend([],[], frameon=False)
ax.set(ylim=(0.5, 4.5))
plt.savefig(os.path.join(path,'FWHM-ratio-comparison.pdf'))